# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [13]:
# imports

import os
import requests
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI


In [14]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2:1b'

In [15]:
# set up environment

load_dotenv(override=True)
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')

if openrouter_api_key and openrouter_api_key.startswith('sk-or-') and len(openrouter_api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")



API key looks good so far


In [16]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""



In [17]:
#prompts
system_prompt = "You are a helpful technical tutor who answers questions about python code, software engineering, data science and LLMs"
user_prompt = "Please give a detailed explanation to the following question: " + question

In [18]:
# messages

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [19]:
openrouter_url = "https://openrouter.ai/api/v1"

openrouter= OpenAI(base_url=openrouter_url, api_key=openrouter_api_key)

In [20]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

In [9]:
response=openrouter.chat.completions.create(model=MODEL_GPT,messages=messages)

result=response.choices[0].message.content
print(result)

The code you've presented uses a combination of Python's `yield` statement and a set comprehension. Let's break down the key components and explain each part.

### Breakdown of the Code

1. **`books`**: 
   - This variable is presumably a collection (like a list) of dictionaries, where each dictionary represents a book. Each book dictionary is expected to contain various keys, one of which is "author".

2. **`book.get("author")`**:
   - The `get` method is being called on each `book` dictionary. This retrieves the value associated with the key "author". If the key does not exist, `get` will return `None` instead of raising an error.

3. **`if book.get("author")`**:
   - This is a conditional check. It ensures that only books with a valid (non-None) author are considered. Books that do not have the "author" key or whose "author" value is `None` will be filtered out of the result.

4. **Set Comprehension `{ ... for book in books if book.get("author") }`**:
   - This part creates a set. T

In [21]:
# streaming function
def stream_response(model_type="openrouter"):
    if model_type == "openrouter":
        client, model = openrouter, MODEL_GPT
    elif model_type == "ollama":
        client, model = ollama, MODEL_LLAMA
    else:
        raise ValueError(f"Unknown model_type '{model_type}'. Choose 'openrouter' or 'ollama'.")

    stream = client.chat.completions.create(model=model, messages=messages, stream=True)

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
# Get gpt-4o-mini to answer, with streaming
stream_response("openrouter")

In [22]:
# Get Llama 3.2 to answer
requests.get("http://localhost:11434").content

b'Ollama is running'

In [ ]:
response=ollama.chat.completions.create(model=MODEL_LLAMA,messages=messages)

result=response.choices[0].message.content
print(result)

In [ ]:
# Get Llama 3.2 to answer, with streaming
stream_response("ollama")

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML

SYSTEM_PROMPT = (
    "You are a helpful technical tutor who answers questions about "
    "Python code, software engineering, data science and LLMs."
)

CHOICE_GPT   = "openrouter"
CHOICE_LLAMA = "ollama"

def get_explanation(question: str, model_choice: str) -> str:
    msgs = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user",   "content": question},
    ]
    try:
        if model_choice == CHOICE_GPT:
            stream = openrouter.chat.completions.create(model=MODEL_GPT, messages=msgs, stream=True)
        else:
            stream = ollama.chat.completions.create(model=MODEL_LLAMA, messages=msgs, stream=True)
        return "".join(chunk.choices[0].delta.content or "" for chunk in stream)
    except Exception as e:
        return f"**Error:** {e}"

# ── gradient background wrapper ───────────────────────────────────────────
display(HTML("""
<style>
  #tutor-wrap {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    border-radius: 14px;
    padding: 20px 24px 24px;
    width: 560px;
    margin-bottom: 4px;
  }
  #tutor-wrap .tutor-title {
    color: #fff;
    font-size: 16px;
    font-weight: 700;
    font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', sans-serif;
    margin-bottom: 14px;
    letter-spacing: .3px;
  }
  /* inactive toggle: solid dark purple, white text */
  .tutor-toggle .widget-toggle-button {
    background: #4c2e8a !important;
    color: #e8d9ff !important;
    border: 2px solid #7c5cbf !important;
    border-radius: 8px !important;
    font-weight: 600 !important;
    font-size: 13px !important;
  }
  /* active toggle: bright white background, deep purple text */
  .tutor-toggle .widget-toggle-button.mod-active {
    background: #ffffff !important;
    color: #4c2e8a !important;
    border: 2px solid #ffffff !important;
    font-weight: 800 !important;
  }
  .tutor-textarea textarea {
    border-radius: 8px !important;
    border: none !important;
    background: rgba(255,255,255,0.92) !important;
    font-size: 13px !important;
    padding: 10px 12px !important;
  }
  .tutor-ask-btn button {
    background: linear-gradient(135deg, #f9a825 0%, #f06292 100%) !important;
    color: #fff !important;
    border: none !important;
    border-radius: 8px !important;
    font-weight: 700 !important;
    font-size: 13px !important;
    letter-spacing: .3px !important;
    box-shadow: 0 2px 8px rgba(240,98,146,0.4) !important;
  }
  .tutor-ask-btn button:hover {
    background: linear-gradient(135deg, #fbc02d 0%, #e91e8c 100%) !important;
  }
  .tutor-loader { color: rgba(255,255,255,0.85); font-size: 13px; }
</style>
<div id="tutor-wrap">
  <div class="tutor-title">🎓 Technical Question Assistant</div>
</div>
"""))

# ── model toggle ──────────────────────────────────────────────────────────
model_toggle = widgets.ToggleButtons(
    options=[(" GPT-4o-mini", CHOICE_GPT), (" Llama 3.2", CHOICE_LLAMA)],
    value=CHOICE_GPT,
    style=widgets.ToggleButtonsStyle(button_width="150px"),
)
model_toggle.add_class("tutor-toggle")

# ── question input ────────────────────────────────────────────────────────
question_box = widgets.Textarea(
    placeholder="Type your technical question here...",
    layout=widgets.Layout(width="100%", height="100px"),
)
question_box.add_class("tutor-textarea")

# ── submit + loader ───────────────────────────────────────────────────────
submit_button = widgets.Button(
    description="Ask ",
    layout=widgets.Layout(width="90px", height="34px"),
)
submit_button.style.button_color = "#f06292"
submit_button.style.text_color   = "#ffffff"

loader_label = widgets.Label(value="")
loader_label.style.text_color = "#ffffff"

output_area = widgets.Output()

# ── logic (identical to before) ───────────────────────────────────────────
def on_submit(_):
    output_area.clear_output()
    question = question_box.value.strip()
    if not question:
        with output_area:
            print("Please enter a question.")
        return

    loader_label.value     = "⏳ Thinking..."
    submit_button.disabled = True

    label  = "GPT-4o-mini" if model_toggle.value == CHOICE_GPT else "Llama 3.2"
    answer = get_explanation(question, model_toggle.value)

    loader_label.value     = ""
    submit_button.disabled = False

    with output_area:
        print(f" Model: {label}")
        print(f" Question: {question}\n")
        display(Markdown(answer))
        print("\n--- End of response ---")

submit_button.on_click(on_submit)

display(
    model_toggle,
    question_box,
    widgets.HBox([submit_button, loader_label],
                 layout=widgets.Layout(align_items="center", margin="6px 0 0 0")),
    output_area,
)

ToggleButtons(_dom_classes=('tutor-toggle',), options=((' GPT-4o-mini', 'openrouter'), (' Llama 3.2', 'ollama'…

Textarea(value='', layout=Layout(height='100px', width='100%'), placeholder='Type your technical question here…

Output()